In [1]:
import pandas as pd
import pyspark.sql.functions as F
from datetime import datetime
from pyspark.sql.types import *
from pyspark import StorageLevel

import numpy as np
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("mode.chained_assignment", None)

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

In [3]:
# !pip install scikit-plot

In [4]:
import sklearn
import scikitplot as skplt
from sklearn.metrics import classification_report, confusion_matrix, precision_score

<hr />
<hr />
<hr />

In [5]:
result_schema = StructType([
                    StructField('experiment_filter', StringType(), True),
                    StructField('undersampling_method', StringType(), True),
                    StructField('undersampling_column', StringType(), True),
                    StructField('filename', StringType(), True),
                    StructField('experiment_id', StringType(), True),
                    StructField('n_covid', IntegerType(), True),
                    StructField('n_not_covid', IntegerType(), True),
                    StructField('model_name', StringType(), True),
                    StructField('model_seed', StringType(), True),
                    StructField('model_maxIter', IntegerType(), True),
                    StructField('model_maxDepth', IntegerType(), True),
                    StructField('model_maxBins', IntegerType(), True),
                    StructField('model_minInstancesPerNode', IntegerType(), True),
                    StructField('model_minInfoGain', FloatType(), True),
                    StructField('model_featureSubsetStrategy', StringType(), True),
                    StructField('model_n_estimators', IntegerType(), True),
                    StructField('model_learning_rate', FloatType(), True),
                    StructField('model_impurity', StringType(), True),
                    StructField('model_AUC_ROC', StringType(), True),
                    StructField('model_AUC_PR', StringType(), True),
                    StructField('model_covid_precision', StringType(), True),
                    StructField('model_covid_recall', StringType(), True),
                    StructField('model_covid_f1', StringType(), True),
                    StructField('model_not_covid_precision', StringType(), True),
                    StructField('model_not_covid_recall', StringType(), True),
                    StructField('model_not_covid_f1', StringType(), True),
                    StructField('model_avg_precision', StringType(), True),
                    StructField('model_avg_recall', StringType(), True),
                    StructField('model_avg_f1', StringType(), True),
                    StructField('model_avg_acc', StringType(), True),
                    StructField('model_TP', StringType(), True),
                    StructField('model_TN', StringType(), True),
                    StructField('model_FN', StringType(), True),
                    StructField('model_FP', StringType(), True),
                    StructField('model_time_exec', StringType(), True),
                    StructField('model_col_set', StringType(), True)
                          ])

In [6]:
# GBT PARAMS

# params_dict = {'maxIter': 20, ===> [100, 200, 300]
#               'maxDepth': 3, ===> [3, 5, 10]
#               'maxBins': 20, ===> [16, 32, 64]
#               'learningRate': 0.01,===> [0.01, 0.1, 0.5]
#               'loss': 'logLoss'}  ===> ["logLoss", "leastSquaresError", "leastAbsoluteError"]

# .addGrid(gbt.seed, [2021]) \
#                             .addGrid(gbt.maxIter, [100, 300, 500]) \
#                             .addGrid(gbt.maxDepth, [3, 5, 10, 15]) \
#                             .addGrid(gbt.maxBins, [50, 200]) \
#                             .addGrid(gbt.minInstancesPerNode, [1, 15]) \
#                             .addGrid(gbt.minInfoGain, [0, 3]) \
#                             .addGrid(gbt.featureSubsetStrategy, ['all', 'auto']).build()


# XGBOOST PARAMS
#     'max_depth': range (2, 10, 1),
#     'n_estimators': range(60, 220, 40),
#     'learning_rate': [0.1, 0.01, 0.05]


# RF PARAMS
#                             .addGrid(rf.seed, [2021]) \
#                             .addGrid(rf.numTrees, range(10, 150, 30)) \
#                             .addGrid(rf.maxDepth, range(3, 15, 2)) \
#                             .addGrid(rf.maxBins, range(10, 90, 30)) \
#                             .addGrid(rf.minInstancesPerNode, range(1, 100, 20)) \
#                             .addGrid(rf.minInfoGain, range(0, 10, 2)) \
#                             .addGrid(rf.featureSubsetStrategy, ['all', 'auto', 'onethird', 'sqrt', 'log2']) \
#                             .addGrid(rf.impurity, ['gini', 'entropy']).build()

<hr />
<hr />
<hr />

In [7]:
cols_sets = {'cols_set_1': ['NU_NOTIFIC', 'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO', 
                            'AGE_GROUP', 'DIST_PRI_NOTIFIC_Q',
                            'DIST_PRI_INTERNA_Q', 'DIST_PRI_ENTUTI_Q', 'DIST_PRI_SAIDUTI_Q', 'DIST_PRI_EVOLUCA_Q', 'DIST_PRI_ENCERRA_Q',
                            'SYMP_GROUP1', 'SYMP_GROUP2', 'SYMP_GROUP3', 'SYMP_GROUP4',
                            'RF_GROUP1', 'RF_GROUP2', 'RF_GROUP3', 'RF_GROUP4',
                            'SUPORT_VEN', 'UTI', 'HOSPITAL',
                            'DIST_PRI_RAIOX_Q', 'DIST_PRI_COLETA_Q', 'DIST_PRI_TOMO_Q', 'DIST_PRI_IF_Q', 'DIST_PRI_TRA_Q', 'DIST_PRI_PCR_Q', 'DIST_PRI_SOR_Q',
                            'GMR_TRANSIT_STATIONS_1WEEK_BEFORE_Q', 'GMR_RETAIL_AND_RECREATION_1WEEK_BEFORE_Q', 'GMR_RESIDENTIAL_PERCENT_1WEEK_BEFORE_Q', 'GMR_WORKPLACES_PERCENT_1WEEK_BEFORE_Q',
                            'GMR_TRANSIT_STATIONS_2WEEKS_Q', 'GMR_RETAIL_AND_RECREATION_2WEEKS_Q', 'GMR_RESIDENTIAL_PERCENT_2WEEKS_Q', 'GMR_WORKPLACES_PERCENT_2WEEKS_Q',
                            'INMET_RELATIVE_AIR_HUMIDITY_1WEEK_BEFORE_Q', 'INMET_RELATIVE_AIR_HUMIDITY_2WEEKS_BEFORE_Q'],
             'cols_set_2': ['CS_SEXO','CS_RACA', 'CRITERIO', 'SURTO_SG', 'SUPORT_VEN', 'EVOLUCAO',
                            'OUTRO_SIN', 'AGE_GROUP', 
                            'SYMP_GROUP1',  'SYMP_GROUP2', 'SYMP_GROUP3', 'SYMP_GROUP4',
                            'RF_GROUP1', 'RF_GROUP2', 'RF_GROUP3',  'RF_GROUP4',
                            'DIST_PRI_EVOLUCA_Q', 'DIST_PRI_ENCERRA_Q', 'DIST_PRI_INTERNA_Q',
                            'DIST_PRI_NOTIFIC_Q', 'DIST_PRI_COLETA_Q', 'DIST_PRI_PCR_Q', 'CLASSI_FIN'],
             'cols_set_3': ['SYMP_GROUP1', 'SYMP_GROUP2', 'SYMP_GROUP3', 'SYMP_GROUP4',
                            'AGE_GROUP', 'CS_SEXO', 'CS_RACA', 'EVOLUCAO', 'CS_ESCOL_N',
                            'UTI', 'SUPORT_VEN', 'DIST_PRI_ENTUTI_Q', 'DIST_PRI_INTERNA_Q',
                            'CLASSI_FIN']}

<hr />
<hr />
<hr />

In [8]:
# super_srag = spark.read.parquet('gs://ai-covid19-datalake/standard/super-srag/super_srag_v2.parquet').persist(StorageLevel.MEMORY_ONLY)
# super_srag.limit(2).toPandas()

In [9]:
# super_srag = super_srag.na.fill('9999')
# super_srag = super_srag.na.fill(9999)

<hr />
<hr />
<hr />

In [10]:
def run_gbt(exp_df, params_dict, cols, filename, exp_name, experiment_filter, 
            undersampling_method, undersampling_column, experiment_id):
    import time
    start_time = time.time()
    
    n_covid = exp_df.filter(F.col('CLASSI_FIN') == 1.0).count()
    n_not_covid = exp_df.filter(F.col('CLASSI_FIN') == 0.0).count()
    
    
    id_cols = ['NU_NOTIFIC', 'CLASSI_FIN']

    labelIndexer = StringIndexer(inputCol="CLASSI_FIN", outputCol="indexedLabel").fit(exp_df)    
    
    input_cols = [x for x in exp_df.columns if x not in id_cols]
    assembler = VectorAssembler(inputCols = input_cols, outputCol= 'features')
    exp_df = assembler.transform(exp_df)
    
    # Automatically identify categorical features, and index them.
    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=20).fit(exp_df)
    
    # Split the data into training and test sets (30% held out for testing)
    (trainingData, testData) = exp_df.randomSplit([0.7, 0.3])
    trainingData = trainingData.persist(StorageLevel.MEMORY_ONLY)
    testData = testData.persist(StorageLevel.MEMORY_ONLY)
    
    # Train a RandomForest model.
    gbt = GBTClassifier(labelCol = "indexedLabel", featuresCol = "indexedFeatures",
                       maxIter = params_dict['maxIter'], maxDepth = params_dict['maxDepth'], maxBins = params_dict['maxBins'])
    
    # Convert indexed labels back to original labels.
    labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

    # Chain indexers and forest in a Pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt, labelConverter])

    # Train model.  This also runs the indexers.
    model = pipeline.fit(trainingData)
    
    # Make predictions.
    predictions = model.transform(testData)    
    
    
    pred = predictions.select(['CLASSI_FIN', 'predictedLabel'])\
                  .withColumn('predictedLabel', F.col('predictedLabel').cast('double'))\
                  .withColumn('predictedLabel', F.when(F.col('predictedLabel') == 1.0, 'covid').otherwise('n-covid'))\
                  .withColumn('CLASSI_FIN', F.when(F.col('CLASSI_FIN') == 1.0, 'covid').otherwise('n-covid'))\
                  .toPandas()

    y_true = pred['CLASSI_FIN'].tolist()
    y_pred = pred['predictedLabel'].tolist()
    
    report = classification_report(y_true, y_pred, output_dict=True)
    
    
    evaluator_ROC = BinaryClassificationEvaluator(labelCol="indexedLabel", rawPredictionCol="prediction", metricName="areaUnderROC")
    accuracy_ROC = evaluator_ROC.evaluate(predictions)


    
    evaluator_PR = BinaryClassificationEvaluator(labelCol="indexedLabel", rawPredictionCol="prediction", metricName="areaUnderPR")
    accuracy_PR = evaluator_PR.evaluate(predictions)
    
    conf_matrix = confusion_matrix(y_true, y_pred)

    result_dict = {}
    
    result_dict['experiment_filter'] = experiment_filter
    result_dict['undersampling_method'] = undersampling_method
    result_dict['undersampling_column'] = undersampling_column
    result_dict['filename'] = filename
    result_dict['experiment_id'] = experiment_id
    result_dict['n_covid'] = n_covid
    result_dict['n_not_covid'] = n_not_covid
    result_dict['model_name'] = 'GBT'
    result_dict['model_maxIter'] = params_dict['maxIter']
    result_dict['model_maxDepth'] = params_dict['maxDepth']
    result_dict['model_maxBins'] = params_dict['maxBins']
    result_dict['model_AUC_ROC'] = accuracy_ROC
    result_dict['model_AUC_PR'] = accuracy_PR
    result_dict['model_covid_precision'] = report['covid']['precision']
    result_dict['model_covid_recall'] = report['covid']['recall']
    result_dict['model_covid_f1'] = report['covid']['f1-score']
    result_dict['model_not_covid_precision'] = report['n-covid']['precision']
    result_dict['model_not_covid_recall'] = report['n-covid']['recall']
    result_dict['model_not_covid_f1'] = report['n-covid']['f1-score']
    result_dict['model_avg_precision'] = report['macro avg']['precision']
    result_dict['model_avg_recall'] = report['macro avg']['recall']
    result_dict['model_avg_f1'] = report['macro avg']['f1-score']
    result_dict['model_avg_acc'] = report['accuracy']
    result_dict['model_TP'] = conf_matrix[0][0]
    result_dict['model_TN'] = conf_matrix[1][1]
    result_dict['model_FN'] = conf_matrix[0][1]
    result_dict['model_FP'] = conf_matrix[1][0]
    result_dict['model_time_exec'] = time.time() - start_time
    result_dict['model_col_set'] = cols
    
    return result_dict

<hr />
<hr />
<hr />

# Running GBT on 10 samples for each experiment
### 3x col sets -> ['cols_set_1', 'cols_set_2', 'cols_set_3']
### 3x model_maxIter -> [100, 200, 300]
### 3x model_maxDepth -> [5, 10, 15]
### 3x model_maxBins -> [16, 32, 64]
Total: 10 * 3 * 3 * 3 * 3 = 810

In [11]:
# lists of params
model_maxIter = [100] # [100, 200, 300]
model_maxDepth = [7] # [5, 10, 15]
model_maxBins = [32] # [16, 32, 64]

In [12]:
experiments = []

### Datasets: strat_samp_lab_agegrp

In [13]:
for id_exp in range(3):
    for col_set in list(cols_sets.keys()):
        str_file = 'gs://ai-covid19-datalake/trusted/experiment_map/datasets/rose_nofilter_nostratsamp_' + col_set + '_' + str(id_exp)
        exp_dataframe = spark.read.parquet(str_file)
        print('read it')
        filename = str_file.split('/')[-1].split('.')[0] # strat_samp_lab_agegrp_0
        experiment_name = filename
        undersampling_method = "ROSE"
        experiment_filter = ""
        undersampling_column = ""
        
        for maxIter in model_maxIter:
            for maxDepth in model_maxDepth:
                for maxBins in model_maxBins:
                    params_dict = {'maxIter': maxIter, 'maxDepth': maxDepth, 'maxBins': maxBins}
                    try:                     
                        model = run_gbt(exp_dataframe, params_dict, col_set, filename, experiment_name, experiment_filter, undersampling_method, undersampling_column, id_exp)
                        experiments.append(model)

                        print(experiment_name, str(params_dict), str(model['model_AUC_PR']), str(model['model_time_exec']))
                    except:
                        print('Something wrong with the exp: {}, {}, {}'.format(filename, params_dict, col_set))

read it
rose_nofilter_nostratsamp_cols_set_1_0 {'maxIter': 100, 'maxDepth': 7, 'maxBins': 32} 0.8241987419852379 112.44620776176453
read it
rose_nofilter_nostratsamp_cols_set_2_0 {'maxIter': 100, 'maxDepth': 7, 'maxBins': 32} 0.784581141864623 95.12148118019104
read it
rose_nofilter_nostratsamp_cols_set_3_0 {'maxIter': 100, 'maxDepth': 7, 'maxBins': 32} 0.753574453623794 98.45837116241455
read it
rose_nofilter_nostratsamp_cols_set_1_1 {'maxIter': 100, 'maxDepth': 7, 'maxBins': 32} 0.8268983217723075 92.93456268310547
read it
rose_nofilter_nostratsamp_cols_set_2_1 {'maxIter': 100, 'maxDepth': 7, 'maxBins': 32} 0.767454460694332 94.37762713432312
read it
rose_nofilter_nostratsamp_cols_set_3_1 {'maxIter': 100, 'maxDepth': 7, 'maxBins': 32} 0.7477833867291901 99.22447609901428
read it
rose_nofilter_nostratsamp_cols_set_1_2 {'maxIter': 100, 'maxDepth': 7, 'maxBins': 32} 0.8061744420650786 97.22171306610107
read it
rose_nofilter_nostratsamp_cols_set_2_2 {'maxIter': 100, 'maxDepth': 7, 'maxBi

In [14]:
for i in range(len(experiments)):
    for d in list(experiments[i].keys()):
        experiments[i][d] = str(experiments[i][d])

In [15]:
experiments

[{'experiment_filter': '',
  'undersampling_method': 'ROSE',
  'undersampling_column': '',
  'filename': 'rose_nofilter_nostratsamp_cols_set_1_0',
  'experiment_id': '0',
  'n_covid': '3827',
  'n_not_covid': '3851',
  'model_name': 'GBT',
  'model_maxIter': '100',
  'model_maxDepth': '7',
  'model_maxBins': '32',
  'model_AUC_ROC': '0.8621679719272152',
  'model_AUC_PR': '0.8241987419852379',
  'model_covid_precision': '0.8397129186602871',
  'model_covid_recall': '0.9054170249355116',
  'model_covid_f1': '0.8713280926768721',
  'model_not_covid_precision': '0.8920510304219823',
  'model_not_covid_recall': '0.8189189189189189',
  'model_not_covid_f1': '0.8539220291216533',
  'model_avg_precision': '0.8658819745411347',
  'model_avg_recall': '0.8621679719272153',
  'model_avg_f1': '0.8626250608992627',
  'model_avg_acc': '0.8631764188297404',
  'model_TP': '1053',
  'model_TN': '909',
  'model_FN': '110',
  'model_FP': '201',
  'model_time_exec': '112.44620776176453',
  'model_col_set'

In [16]:
cols = ['experiment_filter', 'undersampling_method', 'undersampling_column', 'filename', 'experiment_id', 'n_covid', 'n_not_covid', 'model_name', 'model_maxIter', 'model_maxDepth', 'model_maxBins', 'model_AUC_ROC', 'model_AUC_PR', 'model_covid_precision', 'model_covid_recall', 'model_covid_f1', 'model_not_covid_precision', 'model_not_covid_recall', 'model_not_covid_f1', 'model_avg_precision', 'model_avg_recall', 'model_avg_f1', 'model_avg_acc', 'model_TP', 'model_TN', 'model_FN', 'model_FP', 'model_time_exec', 'model_col_set']

In [17]:
intermed_results = spark.createDataFrame(data=experiments).select(cols)
intermed_results.toPandas()

/usr/lib/spark/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


,experiment_filter,undersampling_method,undersampling_column,filename,experiment_id,n_covid,n_not_covid,model_name,model_maxIter,model_maxDepth,model_maxBins,model_AUC_ROC,model_AUC_PR,model_covid_precision,model_covid_recall,model_covid_f1,model_not_covid_precision,model_not_covid_recall,model_not_covid_f1,model_avg_precision,model_avg_recall,model_avg_f1,model_avg_acc,model_TP,model_TN,model_FN,model_FP,model_time_exec,model_col_set
0,,ROSE,,rose_nofilter_nostratsamp_cols_set_1_0,0,3827,3851,GBT,100,7,32,0.8621679719272152,0.8241987419852379,0.8397129186602871,0.9054170249355116,0.8713280926768721,0.8920510304219823,0.8189189189189189,0.8539220291216533,0.8658819745411347,0.8621679719272153,0.8626250608992627,0.8631764188297404,1053,909,110,201,112.44620776176453,cols_set_1
1,,ROSE,,rose_nofilter_nostratsamp_cols_set_2_0,0,3827,3851,GBT,100,7,32,0.8189369469154847,0.784581141864623,0.8054474708171206,0.8567880794701986,0.8303249097472925,0.8375586854460094,0.7810858143607706,0.8083371091980064,0.8215030781315651,0.8189369469154846,0.8193310094726494,0.82,1035,892,173,250,95.12148118019104,cols_set_2
2,,ROSE,,rose_nofilter_nostratsamp_cols_set_3_0,0,3827,3851,GBT,100,7,32,0.7895444255800792,0.753574453623794,0.7752365930599369,0.83446519524618,0.803761242845462,0.8099415204678363,0.7446236559139785,0.7759103641456584,0.7925890567638866,0.7895444255800792,0.7898358034955602,0.7907585004359198,983,831,195,285,98.45837116241455,cols_set_3
3,,ROSE,,rose_nofilter_nostratsamp_cols_set_1_1,1,3827,3851,GBT,100,7,32,0.8628951156968292,0.8268983217723075,0.8447000821692687,0.8946910356832027,0.8689771766694844,0.884871550903901,0.8310991957104558,0.8571428571428571,0.8647858165365849,0.8628951156968292,0.8630600169061707,0.86331569664903,1028,930,121,189,92.93456268310547,cols_set_1
4,,ROSE,,rose_nofilter_nostratsamp_cols_set_2_1,1,3827,3851,GBT,100,7,32,0.8162793833360248,0.767454460694332,0.7881773399014779,0.8586762075134168,0.821917808219178,0.8482228626320846,0.7738825591586328,0.8093492208982586,0.8182001012667812,0.8162793833360248,0.8156335145587184,0.8158477202301904,960,883,158,258,94.37762713432312,cols_set_2
5,,ROSE,,rose_nofilter_nostratsamp_cols_set_3_1,1,3827,3851,GBT,100,7,32,0.7920763530039171,0.7477833867291901,0.7688715953307393,0.8408510638297872,0.8032520325203252,0.8213944603629417,0.7433016421780466,0.780399274047187,0.7951330278468405,0.792076353003917,0.7918256532837561,0.7924528301886793,988,860,187,297,99.22447609901428,cols_set_3
6,,ROSE,,rose_nofilter_nostratsamp_cols_set_1_2,2,3827,3851,GBT,100,7,32,0.8522699328687103,0.8061744420650786,0.8209069946195234,0.9066213921901528,0.8616377571601452,0.8932038834951457,0.797918473547268,0.8428767750801649,0.8570554390573346,0.8522699328687104,0.8522572661201551,0.8528528528528528,1068,920,110,233,97.22171306610107,cols_set_1
7,,ROSE,,rose_nofilter_nostratsamp_cols_set_2_2,2,3827,3851,GBT,100,7,32,0.8121778196744787,0.7611395750743671,0.7807570977917981,0.8623693379790941,0.8195364238410596,0.8492366412213741,0.761986301369863,0.8032490974729242,0.8149968695065861,0.8121778196744786,0.811392760656992,0.8117443868739206,990,890,158,278,97.23781704902649,cols_set_2
8,,ROSE,,rose_nofilter_nostratsamp_cols_set_3_2,2,3827,3851,GBT,100,7,32,0.7783667787959633,0.7209127730777789,0.7461406518010292,0.8108108108108109,0.7771326485037963,0.8106343283582089,0.7459227467811159,0.7769333929369692,0.778387490079619,0.7783667787959634,0.7770330207203827,0.7770330652368186,870,869,203,296,100.85582423210144,cols_set_3


In [18]:
# intermed_results.write.parquet('gs://ai-covid19-datalake/trusted/int_exp_results_gbt_strat_samp_lab_agegrp-0-9.parquet', mode='overwrite')